In [1]:
# Use Google Colab
use_colab = True
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/685'

    !pip install -r "/content/drive/My Drive/685_Project/requirements.txt"

!pip install sentencepiece

from datasets import load_dataset


import sys
import os
import random
import shutil
import copy
import inspect

os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 

import numpy as np
import torch
import transformers
import datasets
import sklearn.metrics
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tqdm
from transformers import RobertaTokenizer, RobertaModel, RobertaForSequenceClassification
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/My Drive/685'
/content


In [2]:
assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")


Found device: Tesla K80, n_gpu: 1


In [3]:
# NOT         0 0 0
# OFF UNT     1 1 0
# OFF TIN IND 1 2 1
# OFF TIN GRP 1 2 2
# OFF TIN OTH 1 2 3

In [14]:
import pandas as pd
train = pd.read_csv('/content/drive/My Drive/685_Project/olid_train_v2.csv',sep='\t')
train  = train[train.subtask_a != "NOT"]
#train['subtask_a'] =train['subtask_a'].map({'NOT':0, 'OFF':1, 'UNT':1, 'TIN':2, 'IND':1, 'GRP':2, 'OTH':3, np.nan:0})
train['subtask_b'] =train['subtask_b'].map({'UNT':0, 'TIN':1})
print(len(train["subtask_a"]))
# train["subtask_a"] =  train["subtask_a"].apply(lambda x: list(map(int, x)))


val = train.sample(frac=0.065,replace=False)
final = pd.concat([train, val, val]).drop_duplicates(keep=False)

#sanity checks
assert len(final) == 4400 - len(val)
s1 = pd.merge(val, final, how='inner', on=['id'])
assert len(s1)==0

s1 = pd.merge(final, train, how='inner', on=['id'])
assert len(s1) == len(final)

s1 = pd.merge(val, train, how='inner', on=['id'])
assert len(s1) == len(val)

val.to_csv('olid_val_v2.csv',sep='\t')
final.to_csv('olid_train_v3.csv',sep='\t')

train_dataset = load_dataset('csv', data_files={'train': './olid_train_v3.csv','valid': 'olid_val_v2.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_a_dataset = load_dataset('csv', data_files={'test': '/content/drive/My Drive/685_Project/olid_test_subtask_a.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_b_dataset = load_dataset('csv', data_files={'test': '/content/drive/My Drive/685_Project/olid_test_subtask_b.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_c_dataset = load_dataset('csv', data_files={'test': '/content/drive/My Drive/685_Project/olid_test_subtask_c.csv'}, delimiter="\t", cache_dir='./olid_cache')

Using custom data configuration default-db68b2be806bab29


4400


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to ./olid_cache/csv/default-db68b2be806bab29/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


...

  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-69e8990fed7492fc
Reusing dataset csv (./olid_cache/csv/default-69e8990fed7492fc/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-609ab11a210e5d0a
Reusing dataset csv (./olid_cache/csv/default-609ab11a210e5d0a/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
set_seed(685)

train_dataloader = torch.utils.data.DataLoader(train_dataset['train'], shuffle=True, batch_size=8)
val_dataloader = torch.utils.data.DataLoader(train_dataset['valid'], shuffle=True, batch_size=8)
test_dataloader = torch.utils.data.DataLoader(test_a_dataset['test'], shuffle=True, batch_size=8)


In [23]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', 
    num_labels = 2, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    cache_dir='./roberta_cache'
)
model.to(device) 

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [17]:
batch_size = 99
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
epochs = 5

In [20]:
os.mkdir("trained_model_offenseval_baseline_subtask_b")

In [21]:
import numpy as np
# function to get validation accuracy
def get_validation_performance(val_dataloader):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    
    total_correct = 0

    for batch in val_dataloader:
      if len(batch) == 0: continue
      inputs = tokenizer(batch['tweet'],padding='longest',return_tensors="pt").to(device)
      labels = batch['subtask_b'].to(device)
      
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        outputs = model(**inputs)
        loss = outputs.loss
        logits = outputs.logits
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct
        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / (len(val_dataloader) * val_dataloader.batch_size)
    return avg_val_accuracy

In [22]:
import random

# training loop

# For each epoch...
for epoch_i in range(0, epochs):
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...

    for batch in train_dataloader:

      if len(batch) == 0: continue
      # print(batch)
      inputs = tokenizer(batch['tweet'],padding='longest',return_tensors="pt").to(device)
      labels = batch['subtask_b'].to(device)
     
      # Clear the previously calculated gradient
      model.zero_grad()        

      # Perform a forward pass (evaluate the model on this training batch).
      outputs = model(**inputs, labels=labels)

      loss = outputs.loss
      logits = outputs.logits

      total_train_loss += loss.item()

      # Perform a backward pass to calculate the gradients.
      loss.backward()

      # Update parameters and take a step using the computed gradient.
      optimizer.step()
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set. Implement this function in the cell above.
    print(f"Total loss: {total_train_loss}")
    val_acc = get_validation_performance(val_dataloader)
    print(f"Validation accuracy: {val_acc}")
    torch.save(model.state_dict(),os.path.join('./trained_model_offenseval_baseline_subtask_b/', 'subtask_b_baseline_{}.pt'.format(epoch_i)))
    
print("")
print("Training complete!")




======== Epoch 1 / 5 ========
Training...
Total loss: 189.1910523660481
Validation accuracy: 0.8541666666666666

======== Epoch 2 / 5 ========
Training...
Total loss: 190.46852497011423
Validation accuracy: 0.8541666666666666

======== Epoch 3 / 5 ========
Training...
Total loss: 190.2135243192315
Validation accuracy: 0.8541666666666666

======== Epoch 4 / 5 ========
Training...
Total loss: 188.12608090043068
Validation accuracy: 0.8541666666666666

======== Epoch 5 / 5 ========
Training...
Total loss: 189.68079207837582
Validation accuracy: 0.8541666666666666

Training complete!


In [28]:
import pandas as pd
test = pd.read_csv('/content/drive/My Drive/685_Project/olid_test_subtask_b.csv',sep='\t')

test['subtask_b'] =test['label'].map({'UNT':0, 'TIN':1})
test.to_csv('olid_test_subtask_b_v2.csv',sep='\t')
test_a_dataset = load_dataset('csv', data_files={'test': 'olid_test_subtask_b_v2.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_dataloader = torch.utils.data.DataLoader(test_a_dataset['test'], shuffle=True)

Using custom data configuration default-e149b262286a77d3


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to ./olid_cache/csv/default-e149b262286a77d3/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
get_validation_performance(test_dataloader)

0.8875